In [117]:
import tensorflow as tf
import numpy as np
import cv2

In [118]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_thunder_3 (1).tflite")
interpreter.allocate_tensors()

In [119]:
lowerjoints_list = [13,14,15,16]

In [120]:
def locating_joints(frame, confidence_threshold):
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [121]:
def knee_distance(frame):
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [122]:
def left_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [123]:
def ankle_distance(frame):
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [124]:
def right_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [125]:
EDGES = {
    (13, 15): 'm',
    (14, 16): 'm',
    (13, 14): 'm',
    (16,15) : 'm'
}

In [126]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    print(keypoints.shape)

    for edge, _ in edges.items():
        e1, e2 = edge
        y1, x1, conf1 = joints_coordinates[e1]
        y2, x2, conf2 = joints_coordinates[e2]
        
        
        if (conf1 > confidence_threshold) & (conf2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [127]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  dis = math.sqrt(dis)
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  ankle_dis = math.sqrt(ankle_dis)
  print("ankle distance", ankle_dis)
  print("knee_distance:",dis)
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  if (dis <= 85.00000000000000 and dis > 73.000000000000 and ankle_dis <= 85.00000000000000  and ankle_dis > 73.000000000000) or (dis < 85.00000000000000  and dis > 80.00000000000000 and ankle_dis >= 85.00000000000000  and ankle_dis <= 88.00000000000000 ):
    print("running loop 1")
    cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 73.000000000000 and ankle_dis <= 73.000000000000) or (dis < 73.000000000000 and ankle_dis > 73.000000000000 and ankle_dis <= 76.000000000000):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis > 85.00000000000000  and ankle_dis > 85.00000000000000 ):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
    cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     #pass
            
   
    
    

In [130]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    
    image = frame.copy()
    image = tf.image.resize_with_pad(np.expand_dims(image, axis=0),256,256)
    input_image = tf.cast(image, dtype=tf.float32)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("keypoints_with_scores")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("Denormalize keypoints with score")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("x , y and confidence threshold of keypoints")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    locating_joints(frame, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame)
    ankle_distance(frame)
    left_slope(frame)
    right_slope(frame)
    insoles(frame)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
keypoints_with_scores
[[[[0.4406899  0.8510395  0.02483064]
   [0.14593992 0.9007941  0.04470736]
   [0.43590537 0.8525567  0.02566236]
   [0.14254384 0.91029835 0.08297315]
   [0.4205949  0.8007146  0.02158704]
   [0.15004635 0.69926524 0.1280013 ]
   [0.14378987 0.6934042  0.12944022]
   [0.6096945  0.59584445 0.06000945]
   [0.52267176 0.8250378  0.01216716]
   [0.8014796  0.5542884  0.13366893]
   [0.80290604 0.5630743  0.14594483]
   [0.17062916 0.6449001  0.04567966]
   [0.1580721  0.70200545 0.06774732]
   [0.18317275 0.68614006 0.29035902]
   [0.18238315 0.67821914 0.26161534]
   [0.83491635 0.45084035 0.48557377]
   [0.8019865  0.5246

In [129]:
cap.release()
cv2.destroyAllWindows()